# Initialization

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from copy import deepcopy
import copy

In [2]:
import argparse
import torch
import numpy as np
import os
import datetime
import torch.nn as nn
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import time
import math

from collections import OrderedDict
from typing import List, Tuple, Union
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

Training on cuda:0


In [3]:
import random
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## Set Hyperparameters

In [4]:
input_size = 889
hidden_size = 100
output_size = input_size
batch_size = 32
lr = 0.01
numrounds = 10
USE_CUDA = torch.cuda.is_available()

# Load Data

In [5]:
train_data = np.load(f'./train_data_diginetica.npy', allow_pickle=True)
valid_data = np.load(f'./test_data_diginetica.npy', allow_pickle=True)

In [6]:
train_data[0].columns

Index(['sessionId', 'userId', 'itemId', 'timeframe', 'time', 'userId2',
       'delta_t_a', 'delta_t_b', 'h_a', 'm_a', 's_a', 'h_b', 'm_b', 's_b'],
      dtype='object')

In [7]:
print(f"len train: {len(train_data)}; len test: {len(valid_data)}")

len train: 45; len test: 45


In [8]:
max_user = len(valid_data)
max_user

45

In [9]:
# concat all train data as one dataframe
train_combined = np.concatenate(train_data)
#convert to dataframe
train_combined = pd.DataFrame(train_combined)
train_combined.shape

(1455, 14)

In [10]:
train_combined[2].nunique()

889

In [11]:
# Step 1: Extract unique item IDs from the combined DataFrame
all_unique_items = train_combined[2].unique()

# Step 2: Create a universal item index mapping
universal_item_map = pd.DataFrame({
    'item_idx': np.arange(len(all_unique_items)),
    'itemId': all_unique_items
})

In [12]:
universal_item_map

,item_idx,itemId
0,0,115599.0
1,1,79898.0
2,2,35039.0
3,3,11604.0
4,4,87524.0
...,...,...
884,884,3694.0
885,885,90072.0
886,886,10440.0
887,887,35015.0


# DataLoader Preparation

In [13]:
class Dataset(object):
    def __init__(self, path, sep=',', session_key='sessionId', item_key='itemId', time_key='time', n_sample=-1, itemmap=None, itemstamp=None, time_sort=False):
        # Read csv
        #self.df = pd.read_csv(path, sep=sep, dtype={session_key: int, item_key: int, time_key: float})
        self.df = path
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        if n_sample > 0:
            self.df = self.df[:n_sample]

        # Add colummn item index to data
        self.add_item_indices(itemmap=itemmap)
        """
        Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        clicks within a session are next to each other, where the clicks within a session are time-ordered.
        """
        self.df.sort_values([session_key, time_key], inplace=True)
        self.click_offsets = self.get_click_offset()
        self.session_idx_arr = self.order_session_idx()

    def add_item_indices(self, itemmap=None):
        """
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # type is numpy.ndarray
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            # Build itemmap is a DataFrame that have 2 columns (self.item_key, 'item_idx)
            itemmap = pd.DataFrame({self.item_key: item_ids,
                                   'item_idx': item2idx[item_ids].values})
        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')

    def get_click_offset(self):
        """
        self.df[self.session_key] return a set of session_key
        self.df[self.session_key].nunique() return the size of session_key set (int)
        self.df.groupby(self.session_key).size() return the size of each session_id
        self.df.groupby(self.session_key).size().cumsum() retunn cumulative sum
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()
        return offsets

    def order_session_idx(self):
        if self.time_sort:
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())
        return session_idx_arr
    
    def __len__(self):
        return len(self.session_idx_arr)

    @property
    def items(self):
        return self.itemmap[self.item_key].unique()

In [14]:
class GRUDataset(Dataset):
    def __init__(self, data, itemmap, session_key='sessionId', item_key='itemId', time_key='time'):
        self.data = data
        self.itemmap = itemmap
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key

        # Map items to indices
        self.data = pd.merge(self.data, self.itemmap, on=self.item_key, how='inner')

        # Sort by session and time
        self.data.sort_values([self.session_key, self.time_key], inplace=True)

        # Group data by session and collect item indices
        self.sessions = self.data.groupby(self.session_key)['item_idx'].apply(list)

    def __len__(self):
        return len(self.sessions)

    def __getitem__(self, index):
        session_items = self.sessions.iloc[index]
        sequence = torch.tensor(session_items[:-1], dtype=torch.long)
        target = torch.tensor(session_items[1:], dtype=torch.long)
        return sequence, target

In [15]:
# class DataLoader():
#     def __init__(self, dataset, batch_size=1):
#         """
#         A class for creating session-parallel mini-batches.

#         Args:
#              dataset (SessionDataset): the session dataset to generate the batches from
#              batch_size (int): size of the batch
#         """
#         self.dataset = dataset
#         self.batch_size = batch_size

#     def __iter__(self):
#         """ Returns the iterator for producing session-parallel training mini-batches.

#         Yields:
#             input (B,): torch.FloatTensor. Item indices that will be encoded as one-hot vectors later.
#             target (B,): a Variable that stores the target item indices
#             masks: Numpy array indicating the positions of the sessions to be terminated
#         """
#         # initializations
#         df = self.dataset.df
#         click_offsets = self.dataset.click_offsets
#         session_idx_arr = self.dataset.session_idx_arr

#         iters = np.arange(self.batch_size)
#         maxiter = iters.max()
#         start = click_offsets[session_idx_arr[iters]]
#         end = click_offsets[session_idx_arr[iters] + 1]
#         mask = []  # indicator for the sessions to be terminated
#         finished = False

#         while not finished:
#             minlen = (end - start).min()
#             # Item indices(for embedding) for clicks where the first sessions start
#             idx_target = df.item_idx.values[start]

#             for i in range(minlen - 1):
#                 # Build inputs & targets
#                 idx_input = idx_target
#                 idx_target = df.item_idx.values[start + i + 1]
#                 input = torch.LongTensor(idx_input)
#                 target = torch.LongTensor(idx_target)
#                 yield input, target, mask

#             # click indices where a particular session meets second-to-last element
#             start = start + (minlen - 1)
#             # see if how many sessions should terminate
#             mask = np.arange(len(iters))[(end - start) <= 1]
#             for idx in mask:
#                 maxiter += 1
#                 if maxiter >= len(click_offsets) - 1:
#                     finished = True
#                     break
#                 # update the next starting/ending point
#                 iters[idx] = maxiter
#                 start[idx] = click_offsets[session_idx_arr[maxiter]]
#                 end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

#     def __len__(self):
#         # Return the number of batches in the dataset
#         return (len(self.dataset) + self.batch_size - 1) // self.batch_size

In [16]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn(batch):
    sequences, targets = zip(*batch)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=-1)
    return sequences_padded, targets_padded

def get_loader(data, itemmap, batch_size=32, shuffle=True):
    dataset = GRUDataset(data, itemmap=itemmap)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)


# Model Architecture

In [17]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, nhead=4, dropout=0.1):
        """
        Initialize the Transformer model.

        Args:
            input_size (int): The number of expected features in the input `x`
            hidden_size (int): The number of features in the hidden state `h`
            output_size (int): The size of the output layer (number of items)
            num_layers (int, optional): Number of transformer layers. Default: 1
            nhead (int, optional): Number of heads in the multiheadattention models. Default: 4
            dropout (float, optional): The dropout value. Default: 0.1
        """
        super(TransformerModel, self).__init__()

        self.hidden_size = hidden_size

        self.model_type = 'Transformer'
        self.src_mask = None

        # Embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)

        # Positional Encoding
        self.pos_encoder = PositionalEncoding(hidden_size, dropout)

        # Transformer layer
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, x):
        if self.src_mask is None or self.src_mask.size(0) != len(x):
            device = x.device
            mask = self._generate_square_subsequent_mask(len(x)).to(device)
            self.src_mask = mask

        x = self.embedding(x) * math.sqrt(self.hidden_size)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, self.src_mask)
        output = self.fc(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

## Loss Function

In [18]:
class TOP1MaxLoss(torch.nn.Module):
    def __init__(self):
        super(TOP1MaxLoss, self).__init__()

    def forward(self, scores, targets):
        # Initialize loss
        loss = 0.0

        # Loop over each element in the batch
        for i in range(scores.size(0)):  # Loop over batch
            for j in range(targets.size(1)):  # Loop over sequence
                if targets[i, j] == -1:  # Skip padding
                    continue

                # Get the score of the target item
                pos_score = scores[i, targets[i, j]]

                # Calculate the difference with all other items
                diff = -torch.sigmoid(pos_score - scores[i])

                # Exclude the positive item from the loss
                diff[targets[i, j]] = 0

                # Add to the total loss
                loss += torch.sum(diff)

        # Average the loss
        loss = loss / (scores.size(0) * targets.size(1))

        return loss

# Train & Test

In [19]:
def evaluate(net, dataloader, device, k):
    """Evaluate the network on the given data loader for top-k recommendation."""
    net.to(device)
    net.eval()
    criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Replace with your loss function
    total_recall = 0.0
    total_mrr = 0.0
    total_count = 0
    total_loss = 0.0

    with torch.no_grad():
        for x, y in dataloader:
            data, target = x.to(device), y.to(device)
            outputs = net(data)
            outputs = outputs.view(-1, outputs.size(-1))  # Flatten output
            target2 = target.view(-1)  # Flatten target

            # Calculate total loss
            total_loss += criterion(outputs, target2).item()

            # Select top-k items
            _, top_k_indices = torch.topk(outputs, k, dim=1)

            # Calculate recall and MRR for each batch
            for i in range(data.size(0)):
                for target_item in target[i]:
                    if target_item == -1:  # Skip padding or any special token
                        continue
                    target_item_scalar = target_item.item()
                    top_k_items = top_k_indices[i].tolist()

                    # Calculate Recall@k
                    if target_item_scalar in top_k_items:
                        total_recall += 1

                    # Calculate MRR@k
                    if target_item_scalar in top_k_items:
                        rank = top_k_items.index(target_item_scalar)
                        total_mrr += 1 / (rank + 1)

                total_count += len(target[i][target[i] != -1])  # Count non-padding elements

    avg_recall = total_recall / total_count
    avg_mrr = total_mrr / total_count
    avg_loss = total_loss / len(dataloader)

    results = {
        'recall': avg_recall,
        'mrr': avg_mrr
    }

    return avg_loss, results


In [20]:
def train(net, trainloader, epochs, device, valloader=None):
    """Train the network for session-based recommendation."""
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Assuming -1 is your padding index
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)  # Adam optimizer is often used for Transformers

    print(f"Training {epochs} epoch(s) w/ {len(trainloader)} batches each")
    start_time = time.time()

    net.to(device)
    net.train()

    for epoch in range(epochs):
        total_loss = 0.0

        for x, y in trainloader:
            data, target = x.to(device), y.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = net(data)
            outputs = outputs.view(-1, outputs.size(-1))  # Flatten output for cross-entropy
            target = target.view(-1)  # Flatten target

            # Compute loss and backpropagate
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Calculate metrics
        val_loss, val_results = evaluate(net, valloader, device, k=3)

        print(f"Epoch {epoch + 1}: Loss: {total_loss / len(trainloader):.4f}, Recall: {val_results['recall']:.4f}, MRR: {val_results['mrr']:.4f}")

        net.train()  # Ensure the network is in training mode

    total_time = time.time() - start_time
    net.to("cpu")  # Move model back to CPU

    print(f"Training completed in {total_time:.2f} seconds")

    return val_results

# Solo Training

In [21]:
#list of recall and mrr for each user
recall_list = []
mrr_list = []

for i in range(max_user):
    print(f"Training on user {i}...")
    local_train = train_data[i]
    local_test = valid_data[i]

    trainloader = get_loader(local_train, itemmap=universal_item_map, batch_size=batch_size)
    testloader = get_loader(local_test, itemmap=universal_item_map, batch_size=batch_size)

    # Initialize the network
    net = TransformerModel(input_size, hidden_size, output_size, num_layers=3)
    net.to(DEVICE)

    # Train the network
    train_res = train(net, trainloader, numrounds, DEVICE, testloader)

    # Evaluate the network
    loss, results = evaluate(net, testloader, DEVICE, k=5)

    print(f"Recall@5: {results['recall']:.4f}")
    print(f"MRR@5: {results['mrr']:.4f}")

    recall_list.append(results['recall'])
    mrr_list.append(results['mrr'])

print(f"Average Recall@5: {np.mean(recall_list):.4f}")
print(f"Average MRR@5: {np.mean(mrr_list):.4f}")

Training on user 0...


d:\anaconda3\envs\tori_rs\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: 7.1845, Recall: 0.5000, MRR: 0.2500
Epoch 2: Loss: 5.4824, Recall: 0.5000, MRR: 0.2500
Epoch 3: Loss: 4.2837, Recall: 0.5000, MRR: 0.2500
Epoch 4: Loss: 3.3533, Recall: 0.5000, MRR: 0.2500
Epoch 5: Loss: 2.8492, Recall: 1.0000, MRR: 0.4167
Epoch 6: Loss: 2.5683, Recall: 1.0000, MRR: 0.7500
Epoch 7: Loss: 2.4650, Recall: 1.0000, MRR: 0.6667
Epoch 8: Loss: 2.3959, Recall: 1.0000, MRR: 0.7500
Epoch 9: Loss: 2.1421, Recall: 0.5000, MRR: 0.5000
Epoch 10: Loss: 2.2083, Recall: 0.5000, MRR: 0.5000
Training completed in 1.00 seconds
Recall@5: 0.5000
MRR@5: 0.5000
Training on user 1...
Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: 7.0084, Recall: 0.0000, MRR: 0.0000


d:\anaconda3\envs\tori_rs\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 2: Loss: 5.5318, Recall: 0.0000, MRR: 0.0000
Epoch 3: Loss: 4.2534, Recall: 0.0000, MRR: 0.0000
Epoch 4: Loss: 3.7853, Recall: 0.0000, MRR: 0.0000
Epoch 5: Loss: 3.1608, Recall: 0.3333, MRR: 0.1667
Epoch 6: Loss: 2.9292, Recall: 0.0000, MRR: 0.0000
Epoch 7: Loss: 2.7409, Recall: 0.0000, MRR: 0.0000
Epoch 8: Loss: 2.4556, Recall: 0.0000, MRR: 0.0000
Epoch 9: Loss: 2.4467, Recall: 0.0000, MRR: 0.0000
Epoch 10: Loss: 2.4962, Recall: 0.3333, MRR: 0.1667
Training completed in 0.16 seconds
Recall@5: 0.3333
MRR@5: 0.1667
Training on user 2...
Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: 7.0329, Recall: 0.5000, MRR: 0.1667
Epoch 2: Loss: 5.1103, Recall: 1.0000, MRR: 0.7500
Epoch 3: Loss: 3.9887, Recall: 1.0000, MRR: 0.7500


...

Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: 6.9056, Recall: 0.0000, MRR: 0.0000
Epoch 2: Loss: 5.2835, Recall: 0.0000, MRR: 0.0000
Epoch 3: Loss: 3.8763, Recall: 0.0000, MRR: 0.0000
Epoch 4: Loss: 3.0720, Recall: 0.0000, MRR: 0.0000
Epoch 5: Loss: 2.8293, Recall: 0.0000, MRR: 0.0000
Epoch 6: Loss: 2.7010, Recall: 0.0000, MRR: 0.0000
Epoch 7: Loss: 2.6474, Recall: 0.0000, MRR: 0.0000
Epoch 8: Loss: 2.6165, Recall: 0.0000, MRR: 0.0000
Epoch 9: Loss: 2.7950, Recall: 0.0000, MRR: 0.0000
Epoch 10: Loss: 2.5113, Recall: 0.0000, MRR: 0.0000
Training completed in 0.15 seconds
Recall@5: 1.0000
MRR@5: 0.2000
Training on user 43...


d:\anaconda3\envs\tori_rs\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: 7.0362, Recall: 0.0000, MRR: 0.0000
Epoch 2: Loss: 5.5289, Recall: 0.0000, MRR: 0.0000
Epoch 3: Loss: 4.2145, Recall: 0.0000, MRR: 0.0000
Epoch 4: Loss: 3.4176, Recall: 1.0000, MRR: 0.5000
Epoch 5: Loss: 3.1090, Recall: 1.0000, MRR: 0.3333
Epoch 6: Loss: 2.6279, Recall: 0.0000, MRR: 0.0000
Epoch 7: Loss: 2.5573, Recall: 0.0000, MRR: 0.0000
Epoch 8: Loss: 2.7104, Recall: 0.0000, MRR: 0.0000
Epoch 9: Loss: 2.1995, Recall: 0.0000, MRR: 0.0000
Epoch 10: Loss: 1.9782, Recall: 0.0000, MRR: 0.0000
Training completed in 0.15 seconds
Recall@5: 0.0000
MRR@5: 0.0000
Training on user 44...


d:\anaconda3\envs\tori_rs\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: 6.7896, Recall: 0.0000, MRR: 0.0000
Epoch 2: Loss: 5.4900, Recall: 0.0000, MRR: 0.0000
Epoch 3: Loss: 4.2956, Recall: 0.0000, MRR: 0.0000
Epoch 4: Loss: 3.2738, Recall: 0.0000, MRR: 0.0000
Epoch 5: Loss: 2.7332, Recall: 0.0000, MRR: 0.0000
Epoch 6: Loss: 2.4611, Recall: 0.0000, MRR: 0.0000
Epoch 7: Loss: 1.8752, Recall: 0.0000, MRR: 0.0000
Epoch 8: Loss: 2.0354, Recall: 0.0000, MRR: 0.0000
Epoch 9: Loss: 1.6604, Recall: 0.0000, MRR: 0.0000
Epoch 10: Loss: 1.6357, Recall: 0.0000, MRR: 0.0000
Training completed in 0.19 seconds
Recall@5: 0.0000
MRR@5: 0.0000
Average Recall@5: 0.5342
Average MRR@5: 0.3162


In [22]:
print(net)

TransformerModel(
  (embedding): Embedding(889, 100)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=100, bias=True)
        (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=100, out_features=889, bias=True)
)


# Centralized

In [23]:
#combine all train data as one dataframe
train_combined = np.concatenate(train_data)
train_combined = pd.DataFrame(train_combined)

#set the column name
train_combined.columns = train_data[0].columns


#combine all test data as one dataframe
test_combined = np.concatenate(valid_data)
test_combined = pd.DataFrame(test_combined)

#set the column name
test_combined.columns = valid_data[0].columns

trainloader = get_loader(train_combined, itemmap=universal_item_map, batch_size=batch_size)
testloader = get_loader(test_combined, itemmap=universal_item_map, batch_size=batch_size)

# Initialize the network
net = TransformerModel(input_size, hidden_size, output_size, num_layers=3)
net.to(DEVICE)

# Train the network
train_res = train(net, trainloader, numrounds, DEVICE, testloader)

# Evaluate the network
loss, results = evaluate(net, testloader, DEVICE, k=5)

print(f"Recall@5: {results['recall']:.4f}")
print(f"MRR@5: {results['mrr']:.4f}")


Training 10 epoch(s) w/ 9 batches each
Epoch 1: Loss: 7.2057, Recall: 0.0000, MRR: 0.0000


d:\anaconda3\envs\tori_rs\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 2: Loss: 6.9820, Recall: 0.0097, MRR: 0.0049
Epoch 3: Loss: 6.7240, Recall: 0.0000, MRR: 0.0000
Epoch 4: Loss: 6.6716, Recall: 0.0000, MRR: 0.0000
Epoch 5: Loss: 6.6440, Recall: 0.0000, MRR: 0.0000
Epoch 6: Loss: 6.5389, Recall: 0.0097, MRR: 0.0049
Epoch 7: Loss: 6.5170, Recall: 0.0097, MRR: 0.0032
Epoch 8: Loss: 6.5183, Recall: 0.0097, MRR: 0.0049
Epoch 9: Loss: 6.5194, Recall: 0.0097, MRR: 0.0049
Epoch 10: Loss: 6.5039, Recall: 0.0000, MRR: 0.0000
Training completed in 1.61 seconds
Recall@5: 0.0097
MRR@5: 0.0019


# FL Settings

In [24]:
hidden_size = 400
lr = 0.001
numrounds = 30
num_layers = 3
nhead = 4

## Client

In [25]:
class Client():
  def __init__(self, client_config:dict):
    # client config as dict to make configuration dynamic
    self.id = client_config["id"]
    self.config = client_config
    self.__model = None

    # check if CUDA is available
    if torch.cuda.is_available():
      self.device = 'cuda'
    else:
       self.device = 'cpu'

    self.train_loader = self.config["train_data"]
    self.valid_loader = self.config["test_data"]

  @property
  def model(self):
    return self.__model

  @model.setter
  def model(self, model):
    self.__model = model

  def __len__(self):
    """Return a total size of the client's local data."""
    return len(self.train_loader.sampler)

  def train(self):
    results = train(net=self.model,
                    trainloader= self.train_loader,
                    epochs= self.config["local_epoch"],
                    device= self.device,
                    valloader= self.valid_loader)
    print(f"Train result client {self.id}: {results}")

  def test(self):
    loss,result = evaluate(net = self.model,
                    dataloader= self.valid_loader,
                    device=self.device, k=5)
    print(f"Test result client {self.id}: {loss, result}")
    return result

## Server

In [26]:
class FedAvg():
  def __init__(self):
    self.globalmodel = TransformerModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers, nhead=nhead, dropout=0.0)
    self.rounds = 0
    self.params = {}

    # check if CUDA is available
    if torch.cuda.is_available():
      self.device = 'cuda'
    else:
       self.device = 'cpu'


  def aggregate(self, round):
    #v1:update the aggregate to save the model with round and date indicator
    modelparams = []
    for i in self.params.keys():
      modelparams.append(self.params[i])

    avg_weights = {}
    for name in modelparams[0].keys():
      avg_weights[name] = torch.mean(torch.stack([w[name] for w in modelparams]), dim = 0)

    self.globalmodel.load_state_dict(avg_weights)

    #current timestamp
    current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    # filename = f"{path_glob_m}/global_model_round_{round}_{current_time}.pth"
    # torch.save(self.globalmodel.state_dict(), filename)

  def clientstrain(self, clientconfig):
    clients = clientconfig
    for i in clients.keys():
      test_client = Client(clients[i])
      test_client.model = copy.deepcopy(self.globalmodel)
      test_client.model.to(self.device)
      test_client.train()
      # test_client.test()
      self.params[i] = test_client.model.state_dict()

  def initiate_FL(self, clientconfig, serverdata):
    clients = clientconfig
    print("Round: {}".format(self.rounds))

    print("Obtaining Weights!!")
    self.clientstrain(clients)

    #### Aggregate model
    print("Aggregating Model!!")
    self.aggregate(self.rounds)

    #### Replace parameters with global model parameters
    for i in self.params.keys():
        self.params[i] = self.globalmodel.state_dict()


    servertest = serverdata
    loss, results = evaluate(net = self.globalmodel,
                    dataloader= servertest,
                    device=self.device, k=5)
    print("Round {} metrics:".format(self.rounds))
    print("Server Loss = {}".format(loss))
    print("Server Recall = {}".format(results['recall']))
    print("Round {} finished!".format(self.rounds))
    self.rounds += 1
    return clients, results['recall']

## Main

In [27]:
clients = {}

for i in range(max_user):
  clients[i] = {"id": i, "val_size": 0.25, "batch_size": batch_size, "local_epoch": 1}
  clients[i]['train_data'] = get_loader(train_data[i], itemmap=universal_item_map, batch_size=batch_size)
  clients[i]['test_data'] = get_loader(valid_data[i], itemmap=universal_item_map, batch_size=batch_size)
  print(f"client: {i}")
  print(f"Number of batches in the dataloader train: {len(clients[i]['train_data'])}")
  print(f"Number of batches in the dataloader test: {len(clients[i]['test_data'])}")

serverdata = get_loader(valid_data[37], itemmap=universal_item_map, batch_size=batch_size)
server = FedAvg() ### initialize server

allrecall = []
for i in range(numrounds):
  clients, recall = server.initiate_FL(clients, serverdata)
  allrecall.append(recall)

print("\n")
print("-" * 50)
print("Recall of all rounds: {}".format(allrecall))

client: 0
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 1
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 2
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 3
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 4
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 5
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 6
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 7
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 8
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 9
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 10

client: 14
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 15
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 16
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 17
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 18
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 19
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 20
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 21
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 22
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 23
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1


d:\anaconda3\envs\tori_rs\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Training 1 epoch(s) w/ 1 batches each
Epoch 1: Loss: 7.0611, Recall: 1.0000, MRR: 0.7500
Training completed in 0.02 seconds
Train result client 2: {'recall': 1.0, 'mrr': 0.75}
Training 1 epoch(s) w/ 1 batches each
Epoch 1: Loss: 6.9337, Recall: 0.3333, MRR: 0.3333
Training completed in 0.02 seconds
Train result client 3: {'recall': 0.3333333333333333, 'mrr': 0.3333333333333333}
Training 1 epoch(s) w/ 1 batches each
Epoch 1: Loss: 6.8168, Recall: 1.0000, MRR: 1.0000
Training completed in 0.02 seconds
Train result client 4: {'recall': 1.0, 'mrr': 1.0}
Training 1 epoch(s) w/ 1 batches each
Epoch 1: Loss: 7.1922, Recall: 0.0000, MRR: 0.0000
Training completed in 0.02 seconds
Train result client 5: {'recall': 0.0, 'mrr': 0.0}
Training 1 epoch(s) w/ 1 batches each
Epoch 1: Loss: 6.7763, Recall: 0.0000, MRR: 0.0000
Training completed in 0.02 seconds
Train result client 6: {'recall': 0.0, 'mrr': 0.0}
Training 1 epoch(s) w/ 1 batches each
Epoch 1: Loss: 6.9973, Recall: 0.0000, MRR: 0.0000
Train

## Test to All Clients

In [28]:
final_model = server.globalmodel
final_model.to(DEVICE)

recall_clients = []
mrr_clients = []
loss_clients = []

# loop for each client
for i in range(max_user):
    print(f"Testing on user {i}...")
    local_test = valid_data[i]
    testloader = get_loader(local_test, itemmap=universal_item_map, batch_size=batch_size)

    # Evaluate the network
    loss, results = evaluate(final_model, testloader, DEVICE, k=5)

    print(f"Recall@5: {results['recall']:.4f}")
    print(f"MRR@5: {results['mrr']:.4f}")

    recall_clients.append(results['recall'])
    mrr_clients.append(results['mrr'])
    loss_clients.append(loss) 

print(f"Average Recall@5: {np.mean(recall_clients):.4f}")
print(f"Average MRR@5: {np.mean(mrr_clients):.4f}")

Testing on user 0...
Recall@5: 0.5000
MRR@5: 0.5000
Testing on user 1...
Recall@5: 0.6667
MRR@5: 0.4000
Testing on user 2...
Recall@5: 1.0000
MRR@5: 0.6250
Testing on user 3...
Recall@5: 0.3333
MRR@5: 0.3333
Testing on user 4...
Recall@5: 1.0000
MRR@5: 1.0000
Testing on user 5...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 6...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 7...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 8...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 9...
Recall@5: 1.0000
MRR@5: 1.0000
Testing on user 10...
Recall@5: 0.6667
MRR@5: 0.6667
Testing on user 11...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 12...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 13...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 14...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 15...
Recall@5: 0.5000
MRR@5: 0.2500
Testing on user 16...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 17...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 18...
Recall@5: 0.0000
MRR@5: 0.0000
Tes